In [1]:
import pandas as pd
import numpy as np
import pickle
import sys
from sklearn.feature_extraction.text import TfidfVectorizer
import nltk
from nltk.stem.porter import *
from nltk.corpus import stopwords
import string
import re
# from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer as VS
# from textstat.textstat import *
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from sklearn import cross_validation
from sklearn.model_selection import StratifiedKFold
import matplotlib.pyplot as plt
import seaborn
%matplotlib inline
# Keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Dense, Flatten, LSTM, Conv1D, MaxPooling1D, Dropout, Activation
from keras.layers.embeddings import Embedding
from keras.models import model_from_json
from keras.optimizers import Adam

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
Using TensorFlow backend.


In [2]:
embeddings_index = dict()
f = open('../glove.6B/glove.6B.300d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

In [3]:
df = pd.read_csv("../Proper Experiments/Parsed Datasets/Davidson.csv")
df.text=df.text.astype(str)
df.head(5)

,Unnamed: 0,text,class
0,0,! ! ! mayasolovely : woman complain cleaning ...,0
1,1,! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...,1
2,2,! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...,1
3,3,! ! ! ! ! ! ! ! ! c g anderson : viva based l...,1
4,4,! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...,1


In [4]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df['text'])
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
vocabulary_size=len(tokenizer.word_index) + 1
print(vocabulary_size)

embedding_matrix = np.zeros((vocabulary_size, 300))
for word, index in tokenizer.word_index.items():
    if index > vocabulary_size - 1:
        break
    else:
        embedding_vector = embeddings_index.get(word)
        if embedding_vector is not None:
            embedding_matrix[index] = embedding_vector

35736


In [5]:
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [6]:
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    break

In [7]:
hidden_size=[75,100,125]
learning_rate=[0.0001,0.001,0.01]
score=[]
accuracy=[]

for i in learning_rate:
    sco=[]
    acc=[]
    for j in hidden_size:
        
        model_glove = Sequential()
        model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
        model_glove.add(LSTM(300))
        model_glove.add(Dense(j, activation='relu'))
        model_glove.add(Dense(1, activation='sigmoid'))
        model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=i), metrics=['accuracy'])
        
        model_glove.fit(Xtrain, np.array(ytrain), epochs = 3)

        pred=model_glove.predict_classes(X_val)
        
        acc.append(accuracy_score(y_val, pred))
        sco.append(f1_score(y_val, pred, average='macro'))
    score.append(sco)
    accuracy.append(acc)

Epoch 1/3
17842/17842 [==============================] - 95s 5ms/step - loss: 0.2394 - acc: 0.9005
Epoch 2/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1638 - acc: 0.9303
Epoch 3/3
17842/17842 [==============================] - 85s 5ms/step - loss: 0.1469 - acc: 0.9378
Epoch 1/3
17842/17842 [==============================] - 84s 5ms/step - loss: 0.2434 - acc: 0.8998
Epoch 2/3
17842/17842 [==============================] - 90s 5ms/step - loss: 0.1624 - acc: 0.9309
Epoch 3/3
17842/17842 [==============================] - 94s 5ms/step - loss: 0.1454 - acc: 0.9386
Epoch 1/3
17842/17842 [==============================] - 84s 5ms/step - loss: 0.2413 - acc: 0.9008
Epoch 2/3
17842/17842 [==============================] - 84s 5ms/step - loss: 0.1618 - acc: 0.9297
Epoch 3/3
17842/17842 [==============================] - 101s 6ms/step - loss: 0.1463 - acc: 0.9387
Epoch 1/3
17842/17842 [==============================] - 108s 6ms/step - loss: 0.1780 - acc: 0.9228
Epoch 2/

In [10]:
print(score)

[[0.8885758148874567, 0.8833340392588722, 0.8887631700503706], [0.8955659970129891, 0.8992551050168056, 0.9072875272122245], [0.9019763820369504, 0.9110753026808777, 0.8956432081432082]]


In [12]:
ac=[]
sc=[]
i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    model_glove = Sequential()
    model_glove.add(Embedding(vocabulary_size, 300, input_length=80, weights=[embedding_matrix], trainable=False))
    model_glove.add(LSTM(300))
    model_glove.add(Dense(100, activation='relu'))
    model_glove.add(Dense(1, activation='sigmoid'))
    model_glove.compile(loss='binary_crossentropy', optimizer=Adam(lr=0.01), metrics=['accuracy'])
    
    print("Fold Number: "+str(i))  
    model_glove.fit(Xtrain, np.array(ytrain), epochs = 5)
    
    pred=model_glove.predict_classes(Xtest)
    
    a=accuracy_score(ytest, pred)
    s=f1_score(ytest, pred, average='macro')
    
    if i==0:
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Davidson.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Davidson.h5")
        print("Saved model to disk")
    elif s>max(sc):
        # serialize model to JSON
        model_json = model_glove.to_json()
        with open("Models/model_Davidson.json", "w") as json_file:
            json_file.write(model_json)
        # serialize weights to HDF5
        model_glove.save_weights("Models/model_Davidson.h5")
        print("Saved model to disk")

    ac.append(a)
    sc.append(s)
    i=i+1    

Fold Number: 0
Epoch 1/5
17842/17842 [==============================] - 94s 5ms/step - loss: 0.1918 - acc: 0.9169
Epoch 2/5
17842/17842 [==============================] - 88s 5ms/step - loss: 0.1333 - acc: 0.9450
Epoch 3/5
17842/17842 [==============================] - 86s 5ms/step - loss: 0.1134 - acc: 0.9549
Epoch 4/5
17842/17842 [==============================] - 83s 5ms/step - loss: 0.0972 - acc: 0.9635
Epoch 5/5
17842/17842 [==============================] - 84s 5ms/step - loss: 0.0874 - acc: 0.9685
Saved model to disk
Fold Number: 1
Epoch 1/5
17842/17842 [==============================] - 84s 5ms/step - loss: 0.1791 - acc: 0.9254
Epoch 2/5
17842/17842 [==============================] - 83s 5ms/step - loss: 0.1230 - acc: 0.9517
Epoch 3/5
17842/17842 [==============================] - 83s 5ms/step - loss: 0.1022 - acc: 0.9608
Epoch 4/5
17842/17842 [==============================] - 84s 5ms/step - loss: 0.0968 - acc: 0.9648
Epoch 5/5
17842/17842 [==============================] - 83

In [13]:
print("Accuracies: ")
print(ac)
print("Mean: "+str(np.mean(np.array(ac))))
print("Std Dev: "+str(np.std(np.array(ac))))

Accuracies: 
[0.9509188704616763, 0.9403854773644106, 0.934304932735426, 0.9473094170403588, 0.9369955156950672]
Mean: 0.9419828426593877
Std Dev: 0.006238594304078398


In [14]:
print("Macro-F1-Scores: ")
print(sc)
print("Mean: "+str(np.mean(np.array(sc))))
print("Std Dev: "+str(np.std(np.array(sc))))

Macro-F1-Scores: 
[0.9135772628439752, 0.8954092926592673, 0.8896583146662371, 0.9057809639791303, 0.8874578739302971]
Mean: 0.8983767416157814
Std Dev: 0.009898653925982643


In [5]:
# load json and create model
json_file = open('Models/model_Davidson.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model_glove = model_from_json(loaded_model_json)
# load weights into new model
model_glove.load_weights("Models/model_Davidson.h5")
print("Loaded model from disk")

Loaded model from disk


In [7]:
# Dataset-1
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Waseem_Hovy.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   mralmubarak yes tried murder way heaven total...      1
1           1   nigzzzk command fear allah quran ritual grove...      1
2           2   ib alkisai islamic documentation based belief...      1
3           3   benfrancisallen : good weeks isis new front o...      0
4           4  self described radical feminist makes death th...      1


In [8]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fe634cb1390> False
<keras.layers.recurrent.LSTM object at 0x7fe634cb14e0> True
<keras.layers.core.Dense object at 0x7fe634cb16a0> True
<keras.layers.core.Dense object at 0x7fe639fcbe80> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fe634cb1390> False
<keras.layers.recurrent.LSTM object at 0x7fe634cb14e0> False
<keras.layers.core.Dense object at 0x7fe634cb16a0> True
<keras.layers.core.Dense object at 0x7fe639fcbe80> True
Epoch 1/8
2841/2841 [==============================] - 6s 2ms/step - loss: 0.9061 - acc: 0.5526
Epoch 2/8
2841/2841 [==============================] - 6s 2ms/step - loss: 0.7023 - acc: 0.6417
Epoch 3/8
2841/2841 [==============================] - 6s 2ms/step - loss: 0.6374 - acc: 0.6853
Epoch 4/8
2841/2841 [==============================] - 7s 3ms/step - loss: 0.6129 - acc: 0.6938
Epoch 5/8
2841/2841 [==============================] - 5s 2ms/step - loss: 

In [9]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7140344706296166, 0.7199155524278676, 0.7163969035890219, 0.7086558761435608, 0.7053854276663146]
Mean: 0.7128776460912762
Std Dev: 0.005238553624223878
Macro-F1-Scores: 
[0.56556709498619, 0.6043435890296533, 0.575327034883721, 0.5339658618392336, 0.5880986138116499]
Mean: 0.5734604389100896
Std Dev: 0.023637494802079627


In [10]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.48716144917340837, 0.47853624208304013, 0.48944405348346237, 0.48346235045742436, 0.48011263639563534]
Mean: 0.4837433463185941
Std Dev: 0.0041123335721767445
Macro-F1-Scores: 
[0.4743997261107729, 0.4672306910774786, 0.4766120159072179, 0.4724835324011135, 0.46916456130483475]
Mean: 0.4719781053602835
Std Dev: 0.0034072511804774225


In [11]:
# Dataset-2
df=pd.read_csv('../Proper Experiments/Parsed Datasets/OffensEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0                 user ask native americans take is       1
1           1   user user home you re drunk ! ! ! user maga t...      1
2           2  amazon investigating chinese employees selling...      0
3           3   user someone should have taken piece shit vol...      1
4           4   user user obama wanted liberals amp; illegals...      0


In [12]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fe63407bc88> False
<keras.layers.recurrent.LSTM object at 0x7fe63407b828> True
<keras.layers.core.Dense object at 0x7fe639f62be0> True
<keras.layers.core.Dense object at 0x7fe63419beb8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fe63407bc88> False
<keras.layers.recurrent.LSTM object at 0x7fe63407b828> False
<keras.layers.core.Dense object at 0x7fe639f62be0> True
<keras.layers.core.Dense object at 0x7fe63419beb8> True
Epoch 1/8
2383/2383 [==============================] - 5s 2ms/step - loss: 0.8051 - acc: 0.5770
Epoch 2/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.6534 - acc: 0.6710
Epoch 3/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.6084 - acc: 0.7109
Epoch 4/8
2383/2383 [==============================] - 4s 2ms/step - loss: 0.5864 - acc: 0.7285
Epoch 5/8
2383/2383 [==============================] - 4s 2ms/step - loss: 

In [13]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.7315436241610739, 0.7284934955937893, 0.7381451951321863, 0.7419219471254721, 0.7331095258078053]
Mean: 0.7346427575640654
Std Dev: 0.0047963010713778925
Macro-F1-Scores: 
[0.6133726204485583, 0.6072558029585964, 0.6371986805379344, 0.6473607731404717, 0.6136723744501746]
Mean: 0.6237720503071471
Std Dev: 0.015618196211725785


In [14]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.5004194630872483, 0.49643306756189676, 0.5148971884179605, 0.502308015107008, 0.5107007973143097]
Mean: 0.5049517062976846
Std Dev: 0.006812667607673811
Macro-F1-Scores: 
[0.4985644858189, 0.49429356014787085, 0.5130944502890239, 0.5015488426889405, 0.5084007416686128]
Mean: 0.5031804161226696
Std Dev: 0.006759954523707878


In [15]:
# Dataset-3
df=pd.read_csv('../Proper Experiments/Parsed Datasets/hatEval_2019.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0  hurray saving many ways potus realdonaldtrump ...      0
1           1  would young fighting age men vast majority one...      0
2           2   kamalaharris illegals dump kids border like r...      0
3           3  times : nearly white states pose an array prob...      0
4           4  orban brussels : european leaders ignoring peo...      0


In [16]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fe5b700fcf8> False
<keras.layers.recurrent.LSTM object at 0x7fe5b700fc50> True
<keras.layers.core.Dense object at 0x7fe5b700fd30> True
<keras.layers.core.Dense object at 0x7fe5b700fc18> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fe5b700fcf8> False
<keras.layers.recurrent.LSTM object at 0x7fe5b700fc50> False
<keras.layers.core.Dense object at 0x7fe5b700fd30> True
<keras.layers.core.Dense object at 0x7fe5b700fc18> True
Epoch 1/8
1618/1618 [==============================] - 3s 2ms/step - loss: 1.5821 - acc: 0.5915
Epoch 2/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.8415 - acc: 0.6644
Epoch 3/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4986 - acc: 0.7892
Epoch 4/8
1618/1618 [==============================] - 3s 2ms/step - loss: 0.4607 - acc: 0.8232
Epoch 5/8
1618/1618 [==============================] - 3s 2ms/step - loss: 

In [17]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8233477455219271, 0.826946847960445, 0.8257107540173053, 0.8263288009888752, 0.826946847960445]
Mean: 0.8258561992897997
Std Dev: 0.0013353558360508241
Macro-F1-Scores: 
[0.4515582655826559, 0.4561527343618702, 0.4591963968235155, 0.45245346869712344, 0.4561527343618702]
Mean: 0.4551027199654071
Std Dev: 0.002776506432122158


In [18]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.5342804200123533, 0.5271940667490729, 0.5550061804697157, 0.5092707045735476, 0.5043263288009888]
Mean: 0.5260155401211357
Std Dev: 0.018226166890794582
Macro-F1-Scores: 
[0.47908622908622905, 0.47155764486088236, 0.4973464157195744, 0.46191451897446895, 0.45199423630150287]
Mean: 0.47237980898853155
Std Dev: 0.015453655521582687


In [19]:
# Dataset-4
df=pd.read_csv('../Proper Experiments/Parsed Datasets/StormfrontWS.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0  class                                               text
0           0      0  march 13th 2014 booklet downloaded 18 300 time...
1           1      0  order help increase booklets downloads would g...
2           2      0  simply copy paste following text youtube video...
3           3      1  click free download colorfully illustrated 132...
4           4      0              click download 7 42 green banner link


In [20]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fe59d4fce80> False
<keras.layers.recurrent.LSTM object at 0x7fe59d4fe358> True
<keras.layers.core.Dense object at 0x7fe59d4fc780> True
<keras.layers.core.Dense object at 0x7fe5b6f5dcc0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fe59d4fce80> False
<keras.layers.recurrent.LSTM object at 0x7fe59d4fe358> False
<keras.layers.core.Dense object at 0x7fe59d4fc780> True
<keras.layers.core.Dense object at 0x7fe5b6f5dcc0> True
Epoch 1/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.8366 - acc: 0.5286
Epoch 2/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.5090 - acc: 0.7871
Epoch 3/8
1926/1926 [==============================] - 4s 2ms/step - loss: 0.4112 - acc: 0.8749
Epoch 4/8
1926/1926 [==============================] - 3s 2ms/step - loss: 0.3771 - acc: 0.8873
Epoch 5/8
1926/1926 [==============================] - 3s 2ms/step - loss: 

In [21]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.8895228215767634, 0.8868120456905504, 0.8847352024922118, 0.8878504672897196, 0.8847352024922118]
Mean: 0.8867311479082914
Std Dev: 0.0018450180708054595
Macro-F1-Scores: 
[0.4888010605787125, 0.4918525618682468, 0.5105743916119136, 0.483761447397811, 0.4782373126241647]
Mean: 0.4906453548161697
Std Dev: 0.010985198861105145


In [22]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.3755186721991701, 0.37694704049844235, 0.37279335410176534, 0.36863966770508827, 0.38317757009345793]
Mean: 0.37541526091958477
Std Dev: 0.0048040304963937325
Macro-F1-Scores: 
[0.3431488398415393, 0.3452134742329377, 0.3395669190757943, 0.33353973920011654, 0.3498604867902869]
Mean: 0.342265891828135
Std Dev: 0.005487730049336821


In [23]:
# Dataset-5
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_1.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

   Unnamed: 0                                               text  class
0           0   ! ! ! mayasolovely : woman complain cleaning ...      0
1           1   ! ! ! ! ! mleew17 : boy dats cold tyga dwn ba...      0
2           2   ! ! ! ! ! ! ! urkindofbrand dawg ! ! ! ! 80sb...      0
3           3   ! ! ! ! ! ! ! ! ! c g anderson : viva based l...      0
4           4   ! ! ! ! ! ! ! ! ! ! ! ! ! shenikaroberts : sh...      0


In [24]:
wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fe59b43cb38> False
<keras.layers.recurrent.LSTM object at 0x7fe59b43c160> True
<keras.layers.core.Dense object at 0x7fe59b43c588> True
<keras.layers.core.Dense object at 0x7fe59b43ca20> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fe59b43cb38> False
<keras.layers.recurrent.LSTM object at 0x7fe59b43c160> False
<keras.layers.core.Dense object at 0x7fe59b43c588> True
<keras.layers.core.Dense object at 0x7fe59b43ca20> True
Epoch 1/8
4460/4460 [==============================] - 10s 2ms/step - loss: 2.0121 - acc: 0.4910
Epoch 2/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2350 - acc: 0.9422
Epoch 3/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2178 - acc: 0.9422
Epoch 4/8
4460/4460 [==============================] - 8s 2ms/step - loss: 0.2077 - acc: 0.9422
Epoch 5/8
4460/4460 [==============================] - 8s 2ms/step - loss:

/home/bhargav/.local/lib/python3.5/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


Fold Number: 2
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fe57eeab278> False
<keras.layers.recurrent.LSTM object at 0x7fe57eeab3c8> True
<keras.layers.core.Dense object at 0x7fe57eeab390> True
<keras.layers.core.Dense object at 0x7fe57ed8bda0> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fe57eeab278> False
<keras.layers.recurrent.LSTM object at 0x7fe57eeab3c8> False
<keras.layers.core.Dense object at 0x7fe57eeab390> True
<keras.layers.core.Dense object at 0x7fe57ed8bda0> True
Epoch 1/8
4461/4461 [==============================] - 9s 2ms/step - loss: 2.0144 - acc: 0.5055
Epoch 2/8
4461/4461 [==============================] - 7s 2ms/step - loss: 0.2265 - acc: 0.9422
Epoch 3/8
4461/4461 [==============================] - 8s 2ms/step - loss: 0.2122 - acc: 0.9422
Epoch 4/8
4461/4461 [==============================] - 9s 2ms/step - loss: 0.2026 - acc: 0.9422
Epoch 5/8
4461/4461 [==============================] - 8s 2ms/step - loss: 

In [25]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))

With Fine Tuning: 
Accuracies: 
[0.941954280591663, 0.9421783953384133, 0.9426008968609866, 0.9419282511210763, 0.9408071748878923]
Mean: 0.9418937997600063
Std Dev: 0.0005944038010372528
Macro-F1-Scores: 
[0.4888827855109345, 0.485114239556889, 0.4890987812757854, 0.48887588003357524, 0.49220609849504215]
Mean: 0.48883555697444525
Std Dev: 0.0022486394544634254


In [26]:
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

Without Fine Tuning: 
Accuracies: 
[0.21223666517256837, 0.23890632003585835, 0.2295964125560538, 0.2226457399103139, 0.22130044843049326]
Mean: 0.22493711722105755
Std Dev: 0.008906457347432645
Macro-F1-Scores: 
[0.2038114271439278, 0.22591707125692706, 0.21796168785094497, 0.2128713700376862, 0.21126826257545664]
Mean: 0.21436596377298853
Std Dev: 0.007342821288132736


In [6]:
# Dataset-6
df=pd.read_csv('../Proper Experiments/Parsed Datasets/GAB.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  joined gab remind retarded jew haters are typi...
1           1      0       left really scared of https : redd it 9rfkts
2           2      0   that literally looks like monkey supposed pre...
3           3      1                                          dumb cunt
4           4      0                                     makes asshole 
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fa22c6f2be0> False
<keras.layers.recurrent.LSTM object at 0x7fa22c6f2f28> True
<keras.layers.core.Dense object at 0x7fa22c6f2e80> True
<keras.layers.core.Dense object at 0x7fa22c49c128> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fa22c6f2be0> False
<keras.layers.recurrent.LSTM object at 0x7fa22c6f2f28> False
<keras.layers.core.Dense object at 0x7fa22c6f2e80> True
<keras.layers.core.Dense object at 0x7fa22c49c128> True
Epoch 1/8
6079/6079 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8613714849531328, 0.8588815789473684, 0.8588583648626419, 0.8476723145254154, 0.8572133574601085]
Mean: 0.8567994201497333
Std Dev: 0.004752974149524578
Macro-F1-Scores: 
[0.857797849692636, 0.8556041097291845, 0.8562879092301185, 0.8443891164153188, 0.8538139669103502]
Mean: 0.8535785903955215
Std Dev: 0.004769635631185838
Without Fine Tuning: 
Accuracies: 
[0.6418352244696596, 0.6476973684210526, 0.6402368810659648, 0.6399078795854581, 0.6422108899490048]
Mean: 0.642377648698228
Std Dev: 0.00280371524871296
Macro-F1-Scores: 
[0.6307201989118467, 0.638142057116793, 0.6289112466430828, 0.6292220306728898, 0.6310782593482733]
Mean: 0.6316147585385771
Std Dev: 0.0033683176288606302


In [8]:
# Dataset-7
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Reddit.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      1  subsection retarded hungarians ohh boy brace l...
1           1      0   hiii got work 444 mainly typa guys imagine wr...
2           2      0                    wow guess soyboys every country
3           3      0   owen benjamin soyboy song goes every country ...
4           4      0   y all hear sumn means live small town for wor...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7fa1bf8a6dd8> False
<keras.layers.recurrent.LSTM object at 0x7fa1bf8a6cc0> True
<keras.layers.core.Dense object at 0x7fa1bfb615f8> True
<keras.layers.core.Dense object at 0x7fa22c4e9eb8> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7fa1bf8a6dd8> False
<keras.layers.recurrent.LSTM object at 0x7fa1bf8a6cc0> False
<keras.layers.core.Dense object at 0x7fa1bfb615f8> True
<keras.layers.core.Dense object at 0x7fa22c4e9eb8> True
Epoch 1/8
4018/4018 [======

In [9]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.8362776810151779, 0.8287705326032853, 0.8432055749128919, 0.8327526132404182, 0.8461921353907417]
Mean: 0.837439707432503
Std Dev: 0.00645482722534297
Macro-F1-Scores: 
[0.7388232844816212, 0.7541136655385692, 0.7577972866223317, 0.7477983579454703, 0.7624106716390492]
Mean: 0.7521886532454083
Std Dev: 0.008216567114533498
Without Fine Tuning: 
Accuracies: 
[0.500373227170938, 0.49004479840716775, 0.5049776007964162, 0.4967645594823295, 0.4967645594823295]
Mean: 0.4977849490678362
Std Dev: 0.004907662264050131
Macro-F1-Scores: 
[0.4980229817793861, 0.48852532711892743, 0.5032199195845294, 0.49511019037221315, 0.49487111509911097]
Mean: 0.4959499067908334
Std Dev: 0.0047779498618335265


In [6]:
# Dataset-8
df=pd.read_csv('../Proper Experiments/Parsed Datasets/Davidson_2.csv')
df.text=df.text.astype(str)
print(df.head(5))
sequences = tokenizer.texts_to_sequences(df['text'])
data = pad_sequences(sequences, maxlen=80)
X_o = data[:]
y_o = df['class'].astype(int)
y_o = np.array(y_o)
X, X_val, y, y_val = cross_validation.train_test_split(X_o, y_o, test_size=0.1, random_state=0, stratify=y_o)
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

wo_ac=[]
wo_sc=[]
ft_ac=[]
ft_sc=[]

i=0
for (train_indices, test_indices) in skf.split(X, y):
    Xtrain, ytrain = X[train_indices], y[train_indices]
    Xtest, ytest = X[test_indices], y[test_indices]
    
    Xft, X_disc, yft, y_disc = cross_validation.train_test_split(Xtrain, ytrain, test_size=0.75, random_state=43, stratify=ytrain)
    
    print("Fold Number: "+str(i))  
    
    # load json and create model
    json_file = open('Models/model_Davidson.json', 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    Davidson_model = model_from_json(loaded_model_json)
    # load weights into new model
    Davidson_model.load_weights("Models/model_Davidson.h5")
    print("Loaded model from disk")

    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    print('Freezing the LSTM Layer')
    # Freeze the layers except the last 2 layers
    for layer in Davidson_model.layers[:-2]:
        layer.trainable = False
    # Check the trainable status of the individual layers
    for layer in Davidson_model.layers:
        print(layer, layer.trainable)
    Davidson_model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    ## Fit fine-tune data
    Davidson_model.fit(Xft, np.array(yft), epochs = 8)

    ftm_pred=Davidson_model.predict_classes(Xtest)
    ft_ac.append(accuracy_score(ytest, ftm_pred))
    ft_sc.append(f1_score(ytest, ftm_pred, average='macro'))

    om_pred=model_glove.predict_classes(Xtest)
    wo_ac.append(accuracy_score(ytest, om_pred))
    wo_sc.append(f1_score(ytest, om_pred, average='macro'))
    
    i=i+1

   Unnamed: 0  class                                               text
0           0      0   ! ! ! mayasolovely : woman complain cleaning ...
1           1      0              momma said pussy cats inside doghouse
2           2      0   addicted2guys : - simplyaddictedtoguys http :...
3           3      0   allaboutmanfeet : http : t co 3gzupfumev woof...
4           4      0   allyhaaaaa : lemmie eat oreo amp; dishes one ...
Fold Number: 0
Loaded model from disk
<keras.layers.embeddings.Embedding object at 0x7ffb4ef728d0> False
<keras.layers.recurrent.LSTM object at 0x7ffb4ef72780> True
<keras.layers.core.Dense object at 0x7ffb4ef72518> True
<keras.layers.core.Dense object at 0x7ffb54669550> True
Freezing the LSTM Layer
<keras.layers.embeddings.Embedding object at 0x7ffb4ef728d0> False
<keras.layers.recurrent.LSTM object at 0x7ffb4ef72780> False
<keras.layers.core.Dense object at 0x7ffb4ef72518> True
<keras.layers.core.Dense object at 0x7ffb54669550> True
Epoch 1/8
1006/1006 [======

In [7]:
print("With Fine Tuning: ")
print("Accuracies: ")
print(ft_ac)
print("Mean: "+str(np.mean(np.array(ft_ac))))
print("Std Dev: "+str(np.std(np.array(ft_ac))))
print("Macro-F1-Scores: ")
print(ft_sc)
print("Mean: "+str(np.mean(np.array(ft_sc))))
print("Std Dev: "+str(np.std(np.array(ft_sc))))
print("Without Fine Tuning: ")
print("Accuracies: ")
print(wo_ac)
print("Mean: "+str(np.mean(np.array(wo_ac))))
print("Std Dev: "+str(np.std(np.array(wo_ac))))
print("Macro-F1-Scores: ")
print(wo_sc)
print("Mean: "+str(np.mean(np.array(wo_sc))))
print("Std Dev: "+str(np.std(np.array(wo_sc))))

With Fine Tuning: 
Accuracies: 
[0.941468253968254, 0.94240317775571, 0.9502982107355865, 0.9512922465208747, 0.9473161033797217]
Mean: 0.9465555984720295
Std Dev: 0.0040036238812854305
Macro-F1-Scores: 
[0.9224679068081761, 0.9246304172774762, 0.9332866472495001, 0.9363839213697318, 0.9297548045062956]
Mean: 0.9293047394422359
Std Dev: 0.005191618885783727
Without Fine Tuning: 
Accuracies: 
[0.9255952380952381, 0.9334657398212513, 0.9254473161033797, 0.9383697813121272, 0.9433399602385686]
Mean: 0.933243607114113
Std Dev: 0.007036235547998616
Macro-F1-Scores: 
[0.906765081303069, 0.9168639231888648, 0.9066946149993507, 0.922953395525338, 0.9287686752106494]
Mean: 0.9164091380454543
Std Dev: 0.008754108554172102
